In [1]:
from utils.PandasToolsFunction import *
from utils.DataPreprocessingTools import *
from utils.loss_functions import mean_absolute_percentage_error

from torch.nn.functional import mse_loss

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

import json

import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [37]:
# Define the linear regression model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)

        self.hidden_layers = torch.nn.ModuleList([torch.nn.Linear(hidden_dim, hidden_dim) for _ in range(n_layers - 1)])

        self.output_layer = nn.Linear(hidden_dim, output_dim)

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.activation = torch.nn.ReLU()

    def forward(self, x): #week, day_numeric, day_type):
        """
        input: week, day_numeric, day_type (normalized)
        output: validations_nb
        """
        x = x.float()

        # normalize
        x[:,0] /= 52.
        x[:,1] /= 7.

        #print("x:", x)

        x = self.activation(self.input_layer(x.float()))
        for layer in self.hidden_layers:
            x = self.activation(layer(x))
        return self.output_layer(x)

In [55]:
class NN_Model():
    def __init__(self, input_dim, hidden_dim, n_layers, train_data, test_size):
        self.train_data = train_data
        self.stations = train_data['station'].unique()
        self.datasets = {}
        self.nns = {}
        self.features_col = ['week', 'day_numeric', 'day_type']

        """
        self.scaler = MinMaxScaler()
        mask = (train_data['station'] == train_data['station'].unique()[0])
        features = train_data[mask][self.features_col]
        _ = self.scaler.fit_transform(features)
        """

        self.loss = mse_loss
        #self.loss = mean_absolute_percentage_error
        #self.loss = torch.nn.SmoothL1Loss

        for station in self.stations :
            # create datasets
            station_indices = train_data.index[train_data['station'] == station]
            features = train_data.loc[station_indices, self.features_col]
            features = features.values
            y = train_data.loc[station_indices, 'y']
            y = np.array(y)
            #X_normalized = self.scaler.transform(features)
            self.datasets[station] = train_test_split(features, y, test_size=test_size, random_state= 42, shuffle=True)

            # creates nns
            self.nns[station] = NeuralNetwork(input_dim, hidden_dim, n_layers, 1)
    
    def train(self, nb_epochs, learning_rate):
        for station in tqdm(self.stations):

            optimizer = optim.Adam(self.nns[station].parameters(), lr=learning_rate)

            X_train, X_test, y_train, y_test = self.datasets[station]

            for epoch in tqdm(range(nb_epochs), leave=False):
                # Forward pass
                X_train, X_test = torch.tensor(X_train, device=device), torch.tensor(X_test, device=device)
                y_train, y_test = torch.tensor(y_train, device=device), torch.tensor(y_test, device=device)
                y_pred = self.nns[station](X_train)
                loss = self.loss(y_pred.float(), y_train.float()) # mean_absolute_percentage_error(y_train, y_pred)

                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            test_pred = self.nns[station](X_test)
            print('Score on test set:', mean_absolute_percentage_error(y_test, test_pred).item())
            print("y pred:", test_pred[0].item(), " y:", y_test[0].item())
            print('y features:', X_test[0])
            print("y pred:", test_pred[1].item(), " y:", y_test[1].item())
            print("y pred:", test_pred[2].item(), " y:", y_test[2].item())
            print("y pred:", test_pred[3].item(), " y:", y_test[3].item())
            print("y pred:", test_pred[4].item(), " y:", y_test[4].item())
            print('y features:', X_test[4])

            break
    
    def predict(self, station, x): #station, week, day_numeric, day_type):
        return self.nn[station](x)


In [56]:
input_dim = 3
n_layers = 2
hidden_dim = 512

train_data = pd.read_csv('data/train_data_day_typed_cov_replaced_all_features.csv')
train_data['date'] = pd.to_datetime(train_data['date'])
train_data = date_filter(train_data, start_date='2019-12-31', end_date='2022-12-31')

test_size = 0.01

nn_model = NN_Model(input_dim, hidden_dim, n_layers, train_data, test_size)

In [57]:
nn_model.train(nb_epochs=1000, learning_rate=0.001)

  0%|          | 0/439 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

/tmp/ipykernel_223375/257267480.py:45: UserWarning: Using a target size (torch.Size([770])) that is different to the input size (torch.Size([770, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = self.loss(y_pred.float(), y_train.float()) # mean_absolute_percentage_error(y_train, y_pred)
/tmp/ipykernel_223375/257267480.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, X_test = torch.tensor(X_train, device=device), torch.tensor(X_test, device=device)
/tmp/ipykernel_223375/257267480.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train, y_test = torch.tensor(y_train, device=device), torch.tensor(y_test, 

Score on test set: 1.0565918684005737
y pred: 129.97715759277344  y: 191
y features: tensor([50,  3,  0])
y pred: 129.7930145263672  y: 139
y pred: 129.78619384765625  y: 183
y pred: 130.21585083007812  y: 41
y pred: 129.7958526611328  y: 40
y features: tensor([44,  6,  1])


In [ ]:
train_data = pd.read_csv('data/train_data_day_typed_cov_replaced_all_features.csv')
train_data['date'] = pd.to_datetime(train_data['date'])
train_data = date_filter(train_data, start_date='2019-12-31', end_date='2022-12-31')

In [48]:
train_data

,date,week,day_name,day_numeric,station,y,day_type
2124,2022-01-01,52,Saturday,5,1J7,26,7
2125,2022-01-01,52,Saturday,5,O2O,15,7
2126,2022-01-01,52,Saturday,5,NZV,15,7
2127,2022-01-01,52,Saturday,5,8QR,23,7
2128,2022-01-01,52,Saturday,5,UMC,39,7
...,...,...,...,...,...,...,...
1099585,2020-12-31,52,Saturday,5,V2P,1227,6
1099586,2020-12-31,52,Saturday,5,N9K,544,6
1099587,2020-12-31,52,Saturday,5,P6E,92,6
1099588,2020-12-31,52,Saturday,5,BDC,91,6


In [37]:
features_columns = train_data[nn_model.features_col]
X_normalized = nn_model.scaler.transform(features_columns)
normalized_train_data = train_data.copy()
normalized_train_data[nn_model.features_col] = X_normalized
y = train_data['y']
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(normalized_train_data, y, test_size=test_size, random_state= 42, shuffle=False)

In [38]:
X_train

,date,week,day_name,day_numeric,station,y,day_type
2124,2022-01-01,1.0,Saturday,0.833333,1J7,26,0.4375
2125,2022-01-01,1.0,Saturday,0.833333,O2O,15,0.4375
2126,2022-01-01,1.0,Saturday,0.833333,NZV,15,0.4375
2127,2022-01-01,1.0,Saturday,0.833333,8QR,23,0.4375
2128,2022-01-01,1.0,Saturday,0.833333,UMC,39,0.4375
...,...,...,...,...,...,...,...
1096203,2020-12-30,1.0,Friday,0.666667,VI4,524,0.0625
1096204,2020-12-30,1.0,Friday,0.666667,8WZ,55,0.0625
1096205,2020-12-30,1.0,Friday,0.666667,FYB,2038,0.0625
1096206,2020-12-30,1.0,Friday,0.666667,5GK,148,0.0625


In [46]:
for index, row in X_train.iterrows():
    x = row[['week', 'day_numeric', 'day_type']]
    x_tensor = torch.tensor([x['week'], x['day_numeric'], x['day_type']])
    print("predicted y:", nn_model.nns[row['station']](x_tensor))
    print("y:", row['y'])

predicted y: tensor([0.8243], grad_fn=<ViewBackward0>)
y: 26
predicted y: tensor([0.7458], grad_fn=<ViewBackward0>)
y: 15
predicted y: tensor([0.7272], grad_fn=<ViewBackward0>)
y: 15
predicted y: tensor([0.9775], grad_fn=<ViewBackward0>)
y: 23
predicted y: tensor([0.6723], grad_fn=<ViewBackward0>)
y: 39
predicted y: tensor([1.3704], grad_fn=<ViewBackward0>)
y: 108
predicted y: tensor([1.1668], grad_fn=<ViewBackward0>)
y: 29
predicted y: tensor([0.1754], grad_fn=<ViewBackward0>)
y: 530
predicted y: tensor([0.9945], grad_fn=<ViewBackward0>)
y: 53
predicted y: tensor([0.7919], grad_fn=<ViewBackward0>)
y: 95
predicted y: tensor([1.1421], grad_fn=<ViewBackward0>)
y: 47
predicted y: tensor([1.3530], grad_fn=<ViewBackward0>)
y: 82
predicted y: tensor([1.2562], grad_fn=<ViewBackward0>)
y: 75
predicted y: tensor([0.4772], grad_fn=<ViewBackward0>)
y: 167
predicted y: tensor([0.4535], grad_fn=<ViewBackward0>)
y: 210
predicted y: tensor([0.1578], grad_fn=<ViewBackward0>)
y: 38
predicted y: tensor(

KeyboardInterrupt: 